In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv("movies.dat", sep = '::', header = None, engine= 'python', encoding='latin-1')
ratings = pd.read_csv("ratings.dat", sep = '::', header = None, engine= 'python', encoding='latin-1')
users = pd.read_csv("users.dat", sep = '::', header = None, engine= 'python', encoding='latin-1')

In [3]:
training_set = pd.read_csv("training_set.csv")
test_set = pd.read_csv("test_set.csv")

In [4]:
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype='int')

In [5]:
training_set.shape

(750121, 4)

In [6]:
# Getting number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(nb_users, nb_movies)

6040 3952


In [7]:
#Converting the data into array with users in rows and movies in columns
def converter(data):
  new_data = []
  for id_users in range(1,nb_users+1):
    id_movies = data[:,1][data[:,0]==id_users]
    id_ratings = data[:,2][data[:,0]==id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies-1] = id_ratings
    new_data.append(list(ratings))
  return new_data

In [8]:
training_set = converter(training_set)
test_set = converter(test_set)

In [9]:
# Converting data into Torch Tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [10]:
# Converting ratings into binary digits
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [23]:
training_set[1 : 1+100].shape

torch.Size([100, 3952])

In [25]:
# Defining RBM class
class RBM():
  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1,nh)
    self.b = torch.randn(1, nv)
  def sample_h(self, x):
    wx = torch.mm(x,self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)

In [26]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv,nh)

In [27]:
#Training the RBM

nb_epoch = 10
for epoch in range(1, nb_epoch+1):
  training_loss = 0
  s = 0.
  for id_user in range(0, nb_users-batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0,vk,ph0,phk)
    training_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
    s += 1.
  print('Epochs: ' + str(epoch) + ' Loss: ' + str(training_loss/s))

Epochs: 1 Loss: tensor(0.2450)
Epochs: 2 Loss: tensor(0.2290)
Epochs: 3 Loss: tensor(0.2281)
Epochs: 4 Loss: tensor(0.2283)
Epochs: 5 Loss: tensor(0.2283)
Epochs: 6 Loss: tensor(0.2281)
Epochs: 7 Loss: tensor(0.2280)
Epochs: 8 Loss: tensor(0.2287)
Epochs: 9 Loss: tensor(0.2278)
Epochs: 10 Loss: tensor(0.2280)


In [28]:
# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
  v = training_set[id_user: id_user + 1]
  vt = test_set[id_user : id_user + 1]
  if len(vt[vt >= 0]) > 0:
    _,h = rbm.sample_h(v)
    _,v = rbm.sample_v(h)
    test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
    s += 1.
  print("Test Loss: " + str(test_loss/s))

Streaming output truncated to the last 5000 lines.
Test Loss: tensor(0.2028)
Test Loss: tensor(0.2031)
Test Loss: tensor(0.2033)
Test Loss: tensor(0.2033)
Test Loss: tensor(0.2034)
Test Loss: tensor(0.2033)
Test Loss: tensor(0.2033)
Test Loss: tensor(0.2031)
Test Loss: tensor(0.2030)
Test Loss: tensor(0.2031)
Test Loss: tensor(0.2032)
Test Loss: tensor(0.2032)
Test Loss: tensor(0.2032)
Test Loss: tensor(0.2034)
Test Loss: tensor(0.2035)
Test Loss: tensor(0.2036)
Test Loss: tensor(0.2035)
Test Loss: tensor(0.2034)
Test Loss: tensor(0.2036)
Test Loss: tensor(0.2036)
Test Loss: tensor(0.2036)
Test Loss: tensor(0.2035)
Test Loss: tensor(0.2035)
Test Loss: tensor(0.2035)
Test Loss: tensor(0.2035)
Test Loss: tensor(0.2036)
Test Loss: tensor(0.2036)
Test Loss: tensor(0.2036)
Test Loss: tensor(0.2038)
Test Loss: tensor(0.2038)
Test Loss: tensor(0.2037)
Test Loss: tensor(0.2037)
Test Loss: tensor(0.2037)
Test Loss: tensor(0.2038)
Test Loss: tensor(0.2036)
Test Loss: tensor(0.2039)
Test Loss: te